In [1]:
%%capture
!pip install stanza
!pip install unidecode

In [2]:
%%capture
import stanza
import unidecode

stanza.download('es')
nlp = stanza.Pipeline('es')

In [80]:
def judeo_parse(s):
  if s.find("h") != -1:
    indices = [i for i, x in enumerate(s) if x == "h"]
    flag_h = 0
    for h in indices:
      string_list = list(s)
      if h != 0:
        if s[h-2] != "c":
          string_list[h] = ""
      else:
        string_list[0] = ""
    s = "".join(string_list)
  if s.find("y") != -1:
    if len(s) == 1:
      s = s.replace("y","i")
  if s.find("ca") != -1:
    s = s.replace("ca","ka")
  if s.find("co") != -1:
    s = s.replace("co","ko")
  if s.find("cu") != -1:
    s = s.replace("cu","ku")
  if s.find("ñ") != -1:
    s = s.replace("ñ","ny")
  if s.find("qu") != -1:
    s = s.replace("qu","k")
  if s.find("rd") != -1:
    s = s.replace("rd","dr")
  if s.find("cr") != -1:
    s = s.replace("cr","kr")
  if s.find("ll") != -1:
    s = s.replace("ll","y")
  if s.find("ge") != -1:
    s = s.replace("ge","dje")
  if s.find("gi") != -1:
    s = s.replace("gi","dji")
  if s.find("cir") != -1:
    s = s.replace("cir","zir")
  if s.find("ción") != -1:
    s = s.replace("ción","sion")
  if s.find("ce") != -1:
    s = s.replace("ce","se")
  if s.find("ci") != -1:
    s = s.replace("ci","si")
  if s.find("gu") != -1:
    s = s.replace("gu","g")
  if s.find("sc") != -1:
    s = s.replace("sc","s")
  if s.find("zar") != -1:
    s = s.replace("zar","sar")
  if s.find("ct") != -1:
    s = s.replace("ct","kt")
  return s

In [81]:
def complete_word(mayor,menor):
  for l in range(len(mayor)-len(menor)):
    menor = menor+" "
  return menor

def get_suffix(a,b):
  flag = 0
  mayor = ""
  menor = ""

  if len(a) > flag:
    mayor = a
    menor = b
    flag = len(a)
  if len(b) > flag:
    mayor = b
    menor = a
  menor = complete_word(mayor,menor)

  f = []
  for x, y in zip(mayor,menor):
    if x == y:
      f.append(x)

  h = ''.join([str(elem) for elem in f])

  for l in range(len(a)-len(h)):
    h = h+" "

  p = []
  for x, y in zip(a,h):
    if x != y:
      p.append(x)

  p = ''.join([str(elem) for elem in p])
  
  return p

In [82]:
def conj_verb(verbo_esp,verbo_ladino):
  verbo_phrase = verbo_esp.text
  verbo_inf = verbo_esp.lemma
  if "ar" in verbo_phrase[-2] or "er" in verbo_phrase[-2] or "ir" in verbo_phrase[-2] or "VerbForm=Inf" in verbo_esp.feats:
    verbo_ladino = verbo_ladino
  elif "ando" in verbo_phrase:
    verbo_ladino = verbo_ladino[:len(verbo_ladino)-2]+"ando"
  elif "iendo" in verbo_phrase:
    verbo_ladino = verbo_ladino+"iendo"
  elif "ado" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ado"
  elif "ido" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ido"
  else:
    suff = get_suffix(verbo_phrase.lower(),verbo_inf)
    if "Number=Sing|Person=1|Tense=Past" in verbo_esp.feats or "Number=Sing|Person=3|Tense=Past" in verbo_esp.feats or "Number=Sing|Person=1|Tense=Pres" in verbo_esp.feats:
      root_ladino = verbo_ladino[:len(verbo_ladino)-2]
      if root_ladino[-1] == suff[:1]:
        verbo_ladino = verbo_ladino[:len(verbo_ladino)-3]+suff
      else:
        verbo_ladino = verbo_ladino[:len(verbo_ladino)-2]+suff
    if "Number=Sing|Person=3|Tense=Fut" in verbo_esp.feats or "Number=Sing|Person=1|Tense=Fut" in verbo_esp.feats or "Number=Plur|Person=3|Tense=Fut" in verbo_esp.feats or "Number=Plur|Person=3|Tense=Past" in verbo_esp.feats or "Number=Plur|Person=1|Tense=Fut" in verbo_esp.feats:
      verbo_ladino = verbo_ladino+suff
    if "Number=Sing|Person=2|Tense=Pres" in verbo_esp.feats or "Number=Plur|Person=3|Tense=Pres" in verbo_esp.feats or "Number=Plur|Person=1|Tense=Pres" in verbo_esp.feats or "|Number=Plur|Person=3|Tense=Imp|" in verbo_esp.feats:
      verbo_ladino = verbo_ladino[:len(verbo_ladino)-1]+suff
  return verbo_ladino

In [83]:
def conj_aux(verbo_esp,verbo_ladino):
  verbo_phrase = verbo_esp.text
  verbo_inf = verbo_esp.lemma
  suff = get_suffix(verbo_phrase.lower(),verbo_inf)
  if "Number=Sing|Person=3|Tense=Imp" in verbo_esp.feats:
    verbo_ladino = verbo_ladino[:len(verbo_ladino)-1]+suff
  elif "|Number=Sing|Person=3|Tense=Pres|" in verbo_esp.feats or "|Number=Sing|Person=1|Tense=Pres|" in verbo_esp.feats or "|Number=Plur|Person=3|Tense=Pres| in verbo_esp.feats":
    verbo_ladino = verbo_ladino[:len(verbo_ladino)-2]+suff
  return verbo_ladino

In [84]:
def conj_adj_sustantivo(word_esp,word_ladino):
  if str(word_esp.feats) != "None":
    if "Sing" in word_esp.feats:
      return word_ladino
    elif "Plur" in word_esp.feats:
      if (word_esp.text[-2:].find("a") != -1 and word_ladino[-2:].find("a") != -1) or (word_esp.text[-2:].find("o") != -1 and word_ladino[-2:].find("o") != -1) :
        suff_word_esp = word_esp.text[len(word_esp.text)-2:]
        #print(word_ladino)
        root_word_lad = word_ladino[:len(word_ladino)-1]
        word_ladino = root_word_lad+suff_word_esp
        #print(suff_word_esp)
        #print(root_word_lad)
        #print(word_ladino)
        return word_ladino
      elif word_esp.text[-1].find("s") != -1:
        if word_esp.text[-2:] == "es" and word_ladino[-1] != "e":
          word_ladino = word_ladino+"es"
        elif word_esp.text[-3:] == "ses" or  word_esp.text[-2:] == "es":
          word_ladino = word_ladino[:len(word_ladino)-1]+"es"
        else:
          word_ladino = word_ladino+"s"
        return word_ladino
      else:
        return word_ladino+"s"
    else:
      return word_ladino
  return word_ladino

In [85]:
def translate(phrase):
  doc = nlp(phrase)
  jud_phrase = ""
  w = ""
  for sent in doc.sentences:
    for word in sent.words:
      flag1 = 0
      flag2 = 0
      #print(word)
      español_word = word.text
      mixed_case = not español_word.islower() and not español_word.isupper()
      if mixed_case == True:
        flag2 = 1
      for d in dic:
        if word.text.lower() == d["src"]:
          ladino_word = d["target"].replace("\n","")
          ladino_word = " ".join(ladino_word.split())
          if word.lemma == ladino_word or word.text.lower() == ladino_word:
            w = español_word
          else:
            w = ladino_word
          flag1 = 1
      if flag1 == 0:
        for d in dic:
          if word.lemma == d["src"]:
            ladino_word = d["target"].replace("\n","")
            ladino_word = " ".join(ladino_word.split())
            if word.lemma == ladino_word or word.text.lower() == ladino_word:
              w = español_word
            if word.upos == "VERB":
              w = conj_verb(word,ladino_word)
              flag2 = 0
            elif word.upos == "NOUN" or word.upos == "ADJ":
              w = conj_adj_sustantivo(word,ladino_word)
            elif word.upos == "AUX":
              w = conj_aux(word,ladino_word)
            elif word.upos == "DET":
              w = español_word
            else:
              w = ladino_word
            flag1 = 1
      if flag1 == 0:
        if word.upos == "PROPN" or word.upos == "DET":
          w = word.text
        else:
          w = judeo_parse(español_word)
      if flag2 == 1:
        jud_phrase += w.replace("\n","").capitalize() + " "
      else:
        jud_phrase += w.replace("\n","") + " "
  s = jud_phrase.replace(" .",".").replace(" ?","?").replace(" !","!").replace(" ,",",").replace("de el","del").replace('" ','"').replace(" ;",";").replace("¿ ","").replace("¡ ","").strip()
  return unidecode.unidecode(corrector_phrase(s))

In [87]:
def corrector_phrase(s):
  filex = open('dic_esp_lad_phr_v2.txt', 'r', encoding="utf-8")
  Lines = filex.readlines()
  for a in Lines:
    if s.find(a.split(";")[0]) != -1:
      s = s.replace(a.split(";")[0],a.split(";")[1].replace("\n",""))
  if s.find("del komida") != -1:
    s = s.replace("del komida","de la komida")
  if s.find("r se") != -1:
    s = s.replace("r se","rse")
  if s.find("r la") != -1:
    s = s.replace("r la","rla")
  if s.find("r las") != -1:
    s = s.replace("r las","rlas")
  if s.find("r lo") != -1:
    s = s.replace("r lo","rlo")
  if s.find("r los") != -1:
    s = s.replace("r las","rlos")
  return s

In [88]:
file1 = open('dic_esp_lad_v3.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:
  p = {"src":l.split(";")[0],"target":l.split(";")[1]}
  dic.append(p)

In [89]:
phrase="mujeres"
translate(phrase)

'mujeres'

Testing

In [91]:
import pandas as pd

df = pd.read_excel("una-fraze-al-dia_lad-tur-eng-spa.xlsx")

esp=[]
lad_correct = []
lad_trans = []

for a in df.index:
  esp.append(df["Español"][a])
  lad_correct.append(df["Ladino"][a])
  lad_trans.append(translate(df["Español"][a]))

p = {'Español':esp,'Ladino':lad_correct,'Ladino_Trans':lad_trans}

df_1 = pd.DataFrame(p)
df_1.head()

df_1.to_excel("translate_esp_ladino.xlsx")

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords
